In [1]:
import torch
from torch import nn # nn contains all of pytorch's building blocks to a neural network
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.utils.data as dataUtil

/home/operation/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.concat([
                # pd.read_csv("./VariablesToSave.1.csv",sep=";"),
                # pd.read_csv("./VariablesToSave.2.csv",sep=";"),
                # pd.read_csv("./VariablesToSave.3.csv",sep=";"),
                pd.read_csv("./VariablesToSave.4.csv",sep=";")
                ])

print(data.corr()["lrMeasuredForce"])

Name                      0.062777
lrSetPointTension         0.952167
lrDeadTimeCompensation         NaN
aMeasuredTension[1]       0.999890
aMeasuredTension[2]       0.999587
lrPIDVelocity            -0.035573
lrSyncVelocity            0.009344
lrTotalTorqueAdditive     0.529664
lrAccelerationRead       -0.058066
lrVelocityRead            0.003743
lrActDiameter            -0.014610
lrMeasuredForce           1.000000
Name: lrMeasuredForce, dtype: float64


In [ ]:
data = data.drop("Name",axis="columns")
data = data.drop("lrSetPointTension",axis="columns")
lrDeadTimeCompensation = data["lrDeadTimeCompensation"]
data = data.drop("lrDeadTimeCompensation",axis="columns")
lrTotalTorqueAdditive = data["lrTotalTorqueAdditive"]
data = data.drop("lrTotalTorqueAdditive",axis="columns")
lrAccelerationRead = data["lrAccelerationRead"]
data = data.drop("lrAccelerationRead",axis="columns")
lrVelocityRead = data["lrVelocityRead"]
data = data.drop("lrVelocityRead",axis="columns")
lrActDiameter = data["lrActDiameter"]
data = data.drop("lrActDiameter",axis="columns")
lrSyncVelocity = data["lrSyncVelocity"]
data = data.drop("lrSyncVelocity",axis="columns")
# data = data.drop("aMeasuredTension[1]",axis="columns")
# data = data.drop("aMeasuredTension[2]",axis="columns")

# data.to_csv("./data.csv",encoding='utf-8', index=False,sep=";")

In [ ]:
y=torch.tensor(data["lrMeasuredForce"],dtype=torch.float32).unsqueeze(dim=1)
data = data.drop("lrMeasuredForce",axis="columns")
X=torch.tensor(list(data.values),dtype=torch.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X[0:10])

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X.detach().numpy(), y.detach().numpy())
print(reg.score(X, y))
y_predicted=reg.predict(X)
print(reg.coef_)
# plt.plot(y,label="real")
# plt.plot(y_predicted,label="predicted")
# plt.legend()
# plt.show()
# 0.99995541631530487

In [ ]:
# tension_1 tension_2 Pid velocity
start=55500
end=150500
history=[]
tension_1 =X[start][0]
tension_2 =X[start][1]
controllerOutput = X[start:end][:,2]
# torque =  X[start:end][:,3]
# acc =  X[start:end][:,4]
# diameter =  X[start:end][:,5]
tension_real =  y[start:end]


for i in range(len(controllerOutput)):
    #create real tensor 
    # input = torch.tensor([tension_1,tension_2,controllerOutput[i],torque[i],acc[i]],dtype=torch.float32)
    input = torch.tensor([tension_1,tension_2,controllerOutput[i]],dtype=torch.float32)
    tension_0 = reg.predict([input.detach().numpy()])[0][0]
    history.append(tension_0)
    tension_2=tension_1
    tension_1=tension_0
plt.plot(history,label="simulated")
plt.plot(tension_real,label="real")
plt.legend()
plt.show()

plt.plot(lrDeadTimeCompensation[start:end].to_numpy(),label="lrDeadTimeCompensation")
plt.legend()
plt.show()

plt.plot(lrTotalTorqueAdditive[start:end].to_numpy(),label="lrTotalTorqueAdditive")
plt.legend()
plt.show()

plt.plot(lrAccelerationRead[start:end].to_numpy(),label="lrAccelerationRead")
plt.legend()
plt.show()

plt.plot(lrVelocityRead[start:end].to_numpy(),label="lrVelocityRead")
plt.legend()
plt.show()

plt.plot(lrActDiameter[start:end].to_numpy(),label="lrActDiameter")
plt.legend()
plt.show()

plt.plot(lrSyncVelocity[start:end].to_numpy(),label="lrSyncVelocity")
plt.legend()
plt.show()